In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

## Задание 1
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Rock, приобретенные сотрудниками Microsoft


In [10]:
%%sql
--
SELECT 
    Artist, Name AS TrackName, Title
FROM
    tracks t 
        LEFT JOIN (SELECT AlbumId, Title, ArtistId  FROM albums)  USING (AlbumId)
        LEFT JOIN (SELECT ArtistId, Name AS Artist  FROM artists) USING (ArtistId),
    (
        SELECT
            TrackId
        FROM
            tracks LEFT JOIN (SELECT GenreId, Name AS Genre FROM genres WHERE Name LIKE '%Rock%') USING (GenreId)
        WHERE
            (Genre IS NOT null)
        --
        INTERSECT
        --
        SELECT 
            TrackId
        FROM
            customers LEFT JOIN (SELECT CustomerId, InvoiceId FROM invoices)      USING (CustomerId)
                      LEFT JOIN (SELECT TrackId, InvoiceId    FROM invoice_items) USING (InvoiceId)
        WHERE
            (Company LIKE '%Microsoft%')
    ) sum
WHERE
    t.TrackId == sum.TrackId
    

 * sqlite:///chinook.db
Done.


Artist,TrackName,Title
Foo Fighters,No Way Back,In Your Honor [Disc 1]
Foo Fighters,Still,In Your Honor [Disc 2]
Foo Fighters,Razor,In Your Honor [Disc 2]
Foo Fighters,My Hero,The Colour And The Shape
Guns N' Roses,Double Talkin' Jive,Use Your Illusion I
Guns N' Roses,The Garden,Use Your Illusion I
Guns N' Roses,Don't Damn Me,Use Your Illusion I
Guns N' Roses,Dead Horse,Use Your Illusion I


## Задание 2
Вывести 20 самых больших по размеру треков жанра Blues и формата MPEG (используйте команду LIMIT 10)


In [7]:
%%sql

SELECT
    TrackId, Name, AlbumId, Composer, Milliseconds, Bytes
FROM
    tracks
        LEFT JOIN (SELECT GenreId, Name AS Genre         FROM genres      WHERE Name LIKE '%Blues%') USING (GenreId)
        LEFT JOIN (SELECT MediaTypeId, Name AS MediaType FROM media_types WHERE Name LIKE '%MPEG%')  USING (MediaTypeId)

WHERE 
    (Genre IS NOT null) AND (MediaType IS NOT null)
ORDER BY
    Bytes DESC
LIMIT
    10

 * sqlite:///chinook.db
Done.


TrackId,Name,AlbumId,Composer,Milliseconds,Bytes
204,Talkin' 'Bout Women Obviously,20,Amos Blakemore/Buddy Guy,589531,19161377
2541,Riviera Paradise,205,Stevie Ray Vaughan,528692,17232776
2584,Title Song,210,Chris Robinson/Rich Robinson,505521,16501316
921,Old Love,73,"Eric Clapton, Robert Cray",472920,15780747
2579,Wiser Time,209,Chris Robinson/Rich Robinson,459990,15161907
196,Stone Crazy,20,Buddy Guy,433397,14184984
891,Layla,72,Clapton/Gordon,430733,14115792
2580,Girl From A Pawnshop,209,Chris Robinson/Rich Robinson,404688,13250848
2576,Sometimes Salvation,209,Chris Robinson/Rich Robinson,389146,12749424
916,Running On Faith,73,Jerry Lynn Williams,378984,12536275


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций


In [6]:
%%sql
--
SELECT
    Company, count(Genre)
FROM
    customers
        LEFT JOIN (SELECT CustomerId, InvoiceId  FROM invoices)      USING (CustomerId)
        LEFT JOIN (SELECT InvoiceId,   TrackId   FROM invoice_items) USING (InvoiceId)
        LEFT JOIN (SELECT TrackId,     GenreId   FROM tracks)        USING (TrackId)
        LEFT JOIN (SELECT GenreId, Name AS Genre FROM genres)        USING (GenreId)
WHERE 
    (Company IS NOT null) AND (Genre LIKE '%Jazz%')
GROUP BY Company

 * sqlite:///chinook.db
Done.


Company,count(Genre)
Apple Inc.,2
Google Inc.,2
JetBrains s.r.o.,3
Microsoft Corporation,2
Telus,2


## Задание 4
Для каждой компании вывести общее число заказов их клиентов


In [5]:
%%sql
SELECT
    Company, count(InvoiceId)
FROM
    customers
        LEFT JOIN (SELECT CustomerId, InvoiceId FROM invoices) USING (CustomerId)
WHERE 
    (Company IS NOT null)
GROUP BY Company

 * sqlite:///chinook.db
Done.


Company,count(InvoiceId)
Apple Inc.,7
Banco do Brasil S.A.,7
Embraer - Empresa Brasileira de Aeronáutica S.A.,7
Google Inc.,7
JetBrains s.r.o.,7
Microsoft Corporation,7
Riotur,7
Rogers Canada,7
Telus,7
Woodstock Discos,7


## Задание 5
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых **все** треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек

In [9]:
%%sql
SELECT
    Genre,
    avg(UnitPrice) AS AvgPrice,
    count(TrackId) AS Num
FROM
    tracks
        LEFT JOIN (SELECT GenreId,     Name AS Genre FROM genres)      USING (GenreId)
        LEFT JOIN (SELECT MediaTypeId, Name AS Type  FROM media_types) USING (MediaTypeId)
GROUP BY 
    Genre,
    Type
HAVING min(UnitPrice) > 1.5

 * sqlite:///chinook.db
Done.


Genre,AvgPrice,Num
Comedy,1.9899999999999995,17
Drama,1.9899999999999978,64
Sci Fi & Fantasy,1.9900000000000004,26
Science Fiction,1.9899999999999995,13
TV Shows,1.990000000000001,93


In [ ]:
,,,,,
